In [1]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("microsoft/trocr-base-printed")

# word2int = {}

# dict_file = open('Data/vocabulary_semantic_modified.txt','r')
# dict_list = dict_file.read().splitlines()
# i=0
# for word in dict_list:
#     if not word in word2int:
#         word2int[word] = i
#         i+=1

# dict_file.close()
# new_tokens = set(word2int) - set(tokenizer.vocab.keys())
# print(len(new_tokens))
# print(len(tokenizer.vocab.keys()))
# tokenizer.add_tokens(list(new_tokens))
# print(len(tokenizer.vocab.keys()))
# tokenizer.save_pretrained('tokenizer_Mach4/')

In [2]:
# from transformers import AutoTokenizer
# print(tokenizer_reloaded.vocab.get("<some_new_token>"))

In [3]:
class SimpleTokenizer:
    def __init__(self, vocab, pad_token='<pad>', start_token='<s>', end_token='</s>'):
        self.vocab = vocab
        self.pad_token_id = vocab[pad_token]
        self.cls_token_id = vocab[start_token]
        self.sep_token_id = vocab[end_token]
        self.inverse_vocab = {v: k for k, v in vocab.items()}

    def tokenize(self, text, max_length=None):
        token_ids = []
        
        token_ids.append(self.cls_token_id)
        
        token_ids.extend(self.vocab[token] if token in self.vocab else self.vocab['<unk>'] for token in text.split())

        token_ids.append(self.sep_token_id)
        
        if max_length is not None:
            padding_length = max(0, max_length - len(token_ids))
            token_ids.extend([self.pad_token_id] * padding_length)
        
        return token_ids

    def detokenize(self, token_ids):
        # Converts a list of token IDs back to a string
        # Assuming that the start and end tokens, if present, should not be included in the detokenized output
        assert isinstance(token_ids, list), "token_ids must be a list"
        tokens = [self.inverse_vocab[token_id] for token_id in token_ids if token_id in self.inverse_vocab and token_id not in [self.cls_token_id, self.sep_token_id, self.pad_token_id]]
        return ' '.join(tokens)
    
    def batch_decode(self, batch_token_ids):
        assert isinstance(batch_token_ids, list), "batch_token_ids must be a list"
        decoded_strings = []
        for token_ids in batch_token_ids:
            decoded_strings.append(self.detokenize(token_ids))
        return decoded_strings

dict_file = open('vocabulary_semantic_modified.txt','r')
dict_list = dict_file.read().splitlines()
print(list(dict_list)[:10])
dict_file.close()
vocab = {tok: i for i, tok in enumerate(dict_list)}
tokenizer = SimpleTokenizer(vocab)
text = "barline tie clef-C2"
token_ids = tokenizer.tokenize(text, max_length=256)
print(token_ids)
detokenized_text = tokenizer.detokenize(token_ids)
print("Detokenized text: ", detokenized_text)

['<s>', '<pad>', '</s>', '<unk>', 'barline', 'clef-C1', 'clef-C2', 'clef-C3', 'clef-C4', 'clef-C5']
[0, 4, 1745, 6, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Detokenized text:  barline tie clef-C2


In [4]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image
# from transformers import AutoTokenizer

def word_separator():
    return '\t'

processor = TrOCRProcessor.from_pretrained("processor_trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("Trained_Semantic")  #Put the path of the model that you want to use for prediction
# tokenizer = AutoTokenizer.from_pretrained('trOCR_ext_tokenizer/')

sample_fullpath = 'DataMach5/Primus' + '/' + '000103290-1_1_1' + '/' + '000103290-1_1_1'
# sample_fullpath = 'DataMach5/Primus' + '/' + '000051650-1_1_2' + '/' + '000051650-1_1_2'
image = Image.open(sample_fullpath + '.png').convert('RGB')
image

sample_gt_fullpath = sample_fullpath + '.semantic'
with open(sample_gt_fullpath, 'r') as sample_gt_file:
    sample_gt_words = sample_gt_file.readline().rstrip().split(word_separator())
    sample_gt_plain = ' '.join(sample_gt_words)
print(sample_gt_plain) #This is the original text
model.config.num_beams

/localscratch/bisman.17919066.0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


clef-G2 keySignature-EbM timeSignature-6/8 note-Eb5_half. barline note-Bb4_quarter. tie note-Bb4_quarter note-Ab4_eighth barline note-G4_half. barline note-Bb4_quarter. tie note-Bb4_quarter note-Ab4_eighth barline note-G4_quarter. tie note-G4_quarter note-Ab4_eighth barline note-Bb4_quarter. note-Ab4_quarter note-G4_eighth barline


4

In [5]:
import torch
device = torch.device("cuda")
model.to(device)
import jiwer
def process_image(image):
    # prepare image
    pixel_values = processor(image, return_tensors="pt").pixel_values
    # print(pixel_values.shape)
    # generate 
#     generated_ids = model.generate(pixel_values,do_sample=True,top_k=50, length_penalty = 12.0)
    generated_ids = model.generate(pixel_values.to(device), num_beams=12, length_penalty=2, early_stopping=False)
#     generated_ids = model.generate(pixel_values, num_beams=4, length_penalty=6, early_stopping=True)
    # decode
    # print(generated_ids)
    generated_text = tokenizer.detokenize(generated_ids[0].tolist())

    return generated_text

# left_half, right_half = split_image_in_half(image)
# gen_text1 = process_image(left_half)
# print(gen_text1)
gen_text2 = process_image(image)
print(gen_text2) #This is the predicted Text
print("Error Rate(SER): ",jiwer.wer(sample_gt_plain, gen_text2)) #This is the error rate (accuracy) of the predicted text.

torch.Size([1, 3, 384, 384])
tensor([[   0,    0,   14,  233, 1773, 1312,    4,  692, 1745,  691,  494,    4,
         1611,    4,  692, 1115,  494,    4, 1622, 1745, 1621,  494,    4,  692,
          501, 1603,    4,    2]], device='cuda:0')
clef-G2 keySignature-EbM timeSignature-6/8 note-Eb5_half. barline note-Bb4_quarter. tie note-Bb4_quarter note-Ab4_eighth barline note-G4_half. barline note-Bb4_quarter. note-Db5_quarter note-Ab4_eighth barline note-G4_quarter. tie note-G4_quarter note-Ab4_eighth barline note-Bb4_quarter. note-Ab4_quarter note-G4_eighth barline
0.07692307692307693


In [9]:

def get_num_labels(text):
    labels = tokenizer(text, 
                        padding="max_length", 
                        max_length=512).input_ids
    print(labels)
    num_labels = sum(1 for label in labels if label != tokenizer.pad_token_id and label != -100)
    return num_labels

print(get_num_labels(gen_text2))
print(get_num_labels(sample_gt_plain))


[0, 50403, 1437, 51016, 1437, 50812, 1437, 50765, 1437, 50319, 1437, 51699, 1437, 51189, 1437, 51445, 1437, 51287, 1437, 51298, 1437, 50319, 3318, 1437, 50401, 1437, 51148, 1437, 51980, 1437, 51980, 3318, 1437, 51252, 1437, 51269, 1437, 50310, 1437, 50964, 1437, 51853, 1437, 50319, 17355, 1437, 51196, 1437, 51980, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 